### Notebook containing EDA results of the <> Dataset

In [23]:
import pandas as pd
import sys, os
import datetime



In [ ]:
# setup path to data
data_dir = '../data/'
data_file = pd.ExcelFile(os.path.join(data_dir,'online_retail_II.xlsx'))

In [10]:
# find all tabs in file
data_sheets = data_file.sheet_names

# read in data sheet-wise
for sheet in data_sheets:
    print sheet
df_0910 = data_file.parse(data_sheets[0])
df_1011 = data_file.parse(data_sheets[1])

# merge both years into single dataframe
df_0911 = df_0910
df_0911 = df_0911.append(df_1011)

Year 2009-2010
Year 2010-2011


In [59]:
print df_0911.describe()
df_1011.head()

           Quantity         Price    Customer ID
count  1.067371e+06  1.067371e+06  824364.000000
mean   9.938898e+00  4.649388e+00   15324.638504
std    1.727058e+02  1.235531e+02    1697.464450
min   -8.099500e+04 -5.359436e+04   12346.000000
25%    1.000000e+00  1.250000e+00   13975.000000
50%    3.000000e+00  2.100000e+00   15255.000000
75%    1.000000e+01  4.150000e+00   16797.000000
max    8.099500e+04  3.897000e+04   18287.000000


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [40]:
def create_date_cols(df_in):
    days, months, year = [], [], []
    ''' function to split up invoice date col '''
    for i in range(len(df_in)):
        row = df_in.iloc[i,:]
        days.append(row.InvoiceDate.day)
        months.append(row.InvoiceDate.month)
        year.append(row.InvoiceDate.year)
    return days, months, year

In [43]:
days, months, year = create_date_cols(df_1011)

In [50]:
''' create abstracted data df '''
df_abst = pd.DataFrame([days,months, year]).T
df_abst.columns=(['Day','Month','Year'])

In [51]:
for invoice in 

,Day,Month,Year
0,1,12,2010
1,1,12,2010
2,1,12,2010
3,1,12,2010
4,1,12,2010
5,1,12,2010
6,1,12,2010
7,1,12,2010
8,1,12,2010
9,1,12,2010
